In [ ]:
import pandas as pd
import numpy as np
import ssl
import math
import matplotlib.pyplot as plt

class DecisionTreeNode:
    """
    Represents a node in the decision tree.

    Attributes:
        attribute (str): The attribute on which the node splits.
        value (any): The value of the attribute for the split.
        children (list): A list of child nodes.
        is_leaf (bool): True if the node is a leaf node.
        label (any): The class label if the node is a leaf.
    """
    def __init__(self, attribute=None, value=None, children=None, is_leaf=False, label=None):
        self.attribute = attribute
        self.value = value
        self.children = children if children is not None else []
        self.is_leaf = is_leaf
        self.label = label

def print_tree(node, depth=0):
    """
    Recursively prints the decision tree.

    Args:
        node (DecisionTreeNode): The current node to print.
        depth (int): The current depth in the tree for indentation.
    """
    indent = "  " * depth
    if node.is_leaf:
        print(f"{indent}Leaf: {node.label}")
    else:
        print(f"{indent}{node.attribute} (Split)")
        for child in node.children:
            print(f"{indent} -> On {node.attribute} = {child.value}:")
            print_tree(child, depth + 1)

def predict(instance, node):
    """
    Predicts the class label for a given instance using the decision tree.

    Args:
        instance (pd.Series): An instance of the dataset.(rows except for class feature(column which is our target))
        node (DecisionTreeNode): The current node in the tree.

    Returns:
        The predicted class label.
    """
    while not node.is_leaf:
        attribute_value = instance[node.attribute]
        found = False
        for child in node.children:
            if child.value == attribute_value:
                node = child
                found = True
                break
        if not found:
            return None  # Return None if no matching child is found
    return node.label

def find_entropy_split(D, a, N, epsilon):
    """
    Calculates the entropy for a split on a given attribute with differential privacy.

    Args:
        D (pd.DataFrame): The dataset.
        a (str): The attribute to split on.
        N (int): The total number of instances, adjusted for differential privacy.
        epsilon (float): The privacy budget for the calculation. This is part of the total privacy budget we define

    Returns:
        The entropy of the split.
    """
    tot = 0
    unique_values = D[a].unique()
    for j in unique_values:
        subtree = D[D[a] == j]
        subtree_count = len(subtree) + np.random.laplace(0, 1/epsilon)
        subtree_entropy = 0
        for i in D['Class'].unique():
            count = len(subtree[subtree['Class'] == i]) + np.random.laplace(0, 1/epsilon)
            p_i = count / subtree_count
            if p_i > 0:
                subtree_entropy -= p_i * math.log(p_i)
            else:
                subtree_entropy = 0
        tot += (subtree_count / N) * subtree_entropy
    return tot

def dp_id3(D, A, epsilon1, d):
    """
    Builds a decision tree using the ID3 algorithm with differential privacy.

    Args:
        D (pd.DataFrame): The dataset.
        A (list): The list of attributes.
        epsilon1 (float): The privacy budget for the entire tree.
        d (int): The maximum depth of the tree.

    Returns:
        The root node of the decision tree.
    """
    N = len(D) + np.random.laplace(0, 1/epsilon1)
    f = max([len(D[a].unique()) for a in A])
    if len(A) == 0 or d == 0 or f * len(D['Class'].unique()) < math.sqrt(2) / epsilon1:
        label_count = {}
        for i in D['Class'].unique():
            label_count[i] = len(D[D['Class'] == i]) + np.random.laplace(0, 1/epsilon1)
        mode = max(label_count, key=label_count.get)
        return DecisionTreeNode(is_leaf=True, label=mode)

    epsilon2 = epsilon1 / (2 * len(A))
    G = {}
    for a in A:
        G[a] = find_entropy_split(D, a, N, epsilon2)
    a_cap = min(G, key=G.get)

    root = DecisionTreeNode(attribute=a_cap)
    for j in D[a_cap].unique():
        subtree = D[D[a_cap] == j]
        A_new = A.copy()
        A_new.remove(a_cap)
        child_node = dp_id3(subtree, A_new, epsilon1, d - 1)
        child_node.value = j
        root.children.append(child_node)
    return root 

def calculate_accuracy(original_file, predictions_file):
    """
    Calculates the accuracy of the decision tree based on the original dataset and the predictions.

    Args:
        original_file (str): The filepath to the original dataset.
        predictions_file (str): The filepath to the predictions dataset.

    Returns:
        The accuracy as a percentage.
    """
    # Load the original dataset and predictions
    original_data = pd.read_csv(original_file)
    predictions = pd.read_csv(predictions_file)

    # The class label is in the first column named 'class'
    correct_predictions = sum(original_data['Class'] == predictions['Predicted_Label'])
    total_predictions = len(predictions)

    # Calculate accuracy
    accuracy = (correct_predictions / total_predictions)*100
    return accuracy

def accuracy_vs_epsilon(data, d, epsilon_values, attributes):
    """
    Calculates and plots the accuracy of the decision tree for different values of epsilon.

    Args:
        data (pd.DataFrame): The dataset.
        d (int): The maximum depth of the tree.
        epsilon_values (list): A list of epsilon values to test.
        attributes (list): The list of attributes.

    Returns:
        A list of accuracies for each epsilon value.
    """
    accuracies = []
    for epsilon in epsilon_values:
        # Adjust epsilon1 for differential privacy
        epsilon1 = epsilon / (2 * (d + 1))

        # Build the decision tree with the adjusted epsilon1
        decision_tree_root = dp_id3(data, attributes, epsilon1, d)

        # Predict labels for each instance(attribute list except for class) in the dataset
        predictions = [predict(instance, decision_tree_root) for _, instance in data.iterrows()]

        # Calculate accuracy
        correct_predictions = sum(data['Class'] == predictions)
        accuracy = (correct_predictions / len(predictions))*100
        accuracies.append(accuracy)

    # Plotting
    plt.plot(epsilon_values, accuracies, marker='o')
    plt.xlabel('Epsilon Value')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs Epsilon for Decision Tree')
    plt.show()

    return accuracies

def main():
    """
    Main function to load the dataset, build the decision tree, and evaluate its performance.
    """
    # Load the breast cancer dataset
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.data'
    columns = ['Class', 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps', 'deg-malig', 'breast', 'breast-quad', 'irradiat']
    ssl._create_default_https_context = ssl._create_unverified_context #ask python to bypass or ignore SSL certificate verification when accessing HTTPS resources
    data = pd.read_csv(url, names=columns)
    data = data.astype({col: 'category' for col in data.columns if data[col].dtype == 'object'})#convert all object types to category to make for easier processing
    
    data.to_csv('breast_cancer.csv', index=False)
    
    # Define the privacy budget and other attributes like privacy and depth
    epsilon = 10.0  # Adjust based on privacy requirements
    d = 4
    epsilon1 = epsilon/(2*(d+1))
    attributes = ['age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps', 'deg-malig', 'breast', 'breast-quad', 'irradiat']

    # Build the decision tree with differential privacy
    decision_tree_root = dp_id3(data, attributes, epsilon1, d)
    print_tree(decision_tree_root)

    # Predict and output the results for each instance in the dataset
    predictions = []
    for index, instance in data.iterrows():
        predicted_label = predict(instance, decision_tree_root)
        predictions.append(predicted_label)
    
    # Create a DataFrame with predictions and save to CSV
    prediction_df = pd.DataFrame(predictions, columns=['Predicted_Label'])
    prediction_df.to_csv('predictions.csv', index=False)
    print('Predictions saved to predictions.csv')
    
    accuracy = calculate_accuracy('breast_cancer.csv', 'predictions.csv')
    print('Accuracy:', accuracy)
    
    epsilon_values = [10, 50, 100, 500, 1000]
    accuracies = accuracy_vs_epsilon(data, d, epsilon_values, attributes)
    print('Accuracies:', accuracies)

if __name__ == "__main__":
    main()


tumor-size (Split)
 -> On tumor-size = 30-34:
  menopause (Split)
   -> On menopause = premeno:
    irradiat (Split)
     -> On irradiat = no:
      breast (Split)
       -> On breast = left:
        Leaf: no-recurrence-events
       -> On breast = right:
        Leaf: no-recurrence-events
     -> On irradiat = yes:
      breast-quad (Split)
       -> On breast-quad = left_low:
        Leaf: no-recurrence-events
       -> On breast-quad = right_up:
        Leaf: no-recurrence-events
       -> On breast-quad = left_up:
        Leaf: recurrence-events
   -> On menopause = lt40:
    breast (Split)
     -> On breast = left:
      Leaf: no-recurrence-events
     -> On breast = right:
      Leaf: no-recurrence-events
   -> On menopause = ge40:
    irradiat (Split)
     -> On irradiat = no:
      breast-quad (Split)
       -> On breast-quad = left_low:
        Leaf: no-recurrence-events
       -> On breast-quad = right_up:
        Leaf: no-recurrence-events
       -> On breast-quad = left_up:

Accuracies: [81.46853146853147, 83.56643356643356, 88.11188811188812, 90.55944055944056, 93.7062937062937]
